In [4]:
import numpy as np
import scipy.optimize as opt
from scipy.io import loadmat
from CostGradient import cost, gradient
from SigmoidFunction import sigmoid
class LogisticRegression:
    '''
    Regularised Logistic Regression
    '''
    def __init__(self,lmbda=0.1,num_iter=100,fit_intercept=True,threshold=0.5):
        self.lmbda = lmbda
        self.num_iter = num_iter
        self.fit_intercept = fit_intercept
        self.threshold = threshold
        return None
    
    def intercept(self,X):
        '''
        This allows the y-intercept to be computed
        '''
        ones = np.ones((len(X),1))
        return np.c_[ones,X]
    
    def fit(self,X,y):
        if self.fit_intercept:
            X = self.intercept(X)
        #p = len(y)
        (m,n) = X.shape
        lmbda = self.lmbda
        self.num_class = np.unique(y).size
        
        # binary logistic regression
        if self.num_class == 2:
            coefs = np.zeros((1,n))
            coefs = opt.fmin_cg(f=cost,x0=coefs,fprime=gradient,args=(X,y.flatten(),lmbda),maxiter=self.num_iter,disp=False)
        
        
        # One vs all (multiple class)
        elif self.num_class > 2:
            coefs = np.zeros((self.num_class,n))
            for i in range(self.num_class):
                digit = i 
                coefs[i] = opt.fmin_cg(f=cost,x0=coefs[i],fprime=gradient,args=(X,(y==digit).flatten(),lmbda),maxiter=self.num_iter,disp=False)
        
        self.coefs = coefs
    
    def predict_proba(self,X):
        if self.fit_intercept:
            X = self.intercept(X)
        self.proba = sigmoid(np.dot(X,self.coefs.T))
        return self.proba
    
    def predict(self,X):
        if self.num_class == 2:
            predict = (self.predict_proba(X)>=self.threshold).astype(int)
        elif self.num_class > 2:
            if self.fit_intercept:
                X = self.intercept(X)
            predict = np.argmax(np.dot(X,self.coefs.T),axis=1)
        return predict
    
    def score(self,X,y):
        pred  = self.predict(X)
        return np.mean(pred==y.flatten())*100
